<a href="https://colab.research.google.com/github/Cru1zzz3/python-parallel-programming-cookbook/blob/main/Python_Parallel_Programming_(Lab_4)_Udartsev_Stanislav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using the concurrent.futures Python
modules**

Используя пула потоков или процессов (pooling) указывает на специального менеджера, который используется для
оптимизации и упрощения использования потоков и/или процессов в программе. С его помощью можно отправить задачу (или задачи), которые должны быть выполнены, менеджеру. Пул оснащен внутренней очередью ожидающих выполнения задач и рядом потоков или
процессов, которые их выполняют.

Из вывода можно увидеть, что наименьшее время занимает использования пула процессов.

In [ ]:
import concurrent.futures
import time

number_list = [1,2,3,4,5,6,7,8,9,10]

def evaluate_item(x):
  #count...just to make an operation
  result_item = count(x)
  #print the input item and the result
  print ("item " + str(x) + " result " + str(result_item))

def count(number) :
  for i in range(0,10000000):
    i=i+1
  return i*number

if __name__ == "__main__":
  ##Sequential Execution
  print("\nSequential Execution:")
  start_time = time.process_time()
  for item in number_list:
    evaluate_item(item)
    print ("Sequential execution in " + str(time.process_time() - start_time), "seconds")

  ##Thread pool Execution
  print("\nThread pool Execution:")
  start_time_1 = time.process_time()
  with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    for item in number_list:
      executor.submit(evaluate_item, item)
      print ("Thread pool execution in " + str(time.process_time() - start_time_1), "seconds")

  ##Process pool Execution
  print("\nProcess pool Execution:")
  start_time_2 = time.process_time()
  with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
    for item in number_list:
      executor.submit(evaluate_item, item)
      print ("Process pool execution in " + str(time.process_time() - start_time_2), "seconds")


Sequential Execution:
item 1 result 10000000
Sequential execution in 0.6619279889999987 seconds
item 2 result 20000000
Sequential execution in 1.3247391139999962 seconds
item 3 result 30000000
Sequential execution in 1.9858085959999983 seconds
item 4 result 40000000
Sequential execution in 2.660087007999998 seconds
item 5 result 50000000
Sequential execution in 3.3117808319999966 seconds
item 6 result 60000000
Sequential execution in 3.971531376999998 seconds
item 7 result 70000000
Sequential execution in 4.636356452000001 seconds
item 8 result 80000000
Sequential execution in 5.307656193999996 seconds
item 9 result 90000000
Sequential execution in 5.971374175000001 seconds
item 10 result 100000000
Sequential execution in 6.635407960000002 seconds

Thread pool Execution:
Thread pool execution in 0.005431237999999894 seconds
Thread pool execution in 0.022353432000002726 seconds
Thread pool execution in 0.12701058300000057 seconds
Thread pool execution in 0.14444479700000556 seconds
Thr

**Event loop management with Asyncio**

Для любой функции asyncio для запуска в colab нельзя вызвать цикл.run_forever(...), так как цикл, который вы получите от asyncio.get_event_loop(), будет активным (running=True). И при вызове loop.run_forever() отобразиться ошибка "**RuntimeError: This event loop is already running**"

Выходом из этой ситауции будет запуск этого скрипта напрямую через python в colab из отдельного файла:

In [ ]:
def asyncio_loop():
  """
import asyncio
import datetime
import time

def function_1(end_time, loop):
  print ("function_1 called")
  if (loop.time() + 1.0) < end_time:
    loop.call_later(1, function_2, end_time, loop)
  else:
    loop.stop()


def function_2(end_time, loop):
  print ("function_2 called ")
  if (loop.time() + 1.0) < end_time:
    loop.call_later(1, function_3, end_time, loop)
  else:
    loop.stop()

def function_3(end_time, loop):
  print ("function_3 called")
  if (loop.time() + 1.0) < end_time:
    loop.call_later(1, function_1, end_time, loop)
  else:
    loop.stop()

def function_4(end_time, loop):
  print ("function_5 called")
  if (loop.time() + 1.0) < end_time:
    loop.call_later(1, function_4, end_time, loop)
  else:
    loop.stop()

loop = asyncio.get_event_loop()

end_loop = loop.time() + 9.0
loop.call_soon(function_1, end_loop, loop)
#loop.call_soon(function_4, end_loop, loop)

loop.run_forever()
loop.close()
  """

# asyncio_loop.py
with open('/content/asyncio_loop.py', 'w') as f:
  f.write(asyncio_loop.__doc__)


!python '/content/asyncio_loop.py'

function_1 called
function_2 called 
function_3 called
function_1 called
function_2 called 
function_3 called
function_1 called
function_2 called 
function_3 called


**Handling coroutines with Asyncio**

Корутины(coroutines) вычисляют один вычислительный шаг, а также не используют  функцию для координации результатов между собой, в отличии от подпрограмм. Корутины соединяются вместе, образуя конвейер, без какой-либо надзорной функции, ответственной за их вызов в определенном порядке. В корутине точка выполнения может быть приостановлена и возобновлена позже после отслеживания ее локального состояния за прошедшее время. Имея пул корутин, можно чередовать их вычисления: запускать первую, пока она не вернет управление, затем запускать вторую и так далее. 

In [ ]:
def asyncio_state_machine():
  """
import asyncio
import time
from random import randint

@asyncio.coroutine
def StartState():
  print ("Start State called \\n")
  input_value = randint(0,1)
  time.sleep(1)
  if (input_value == 0):
    result = yield from State2(input_value)
  else :
    result = yield from State1(input_value)
  print("Resume of the Transition : \\nStart State calling " + result)

@asyncio.coroutine
def State1(transition_value):
  outputValue = str(("State 1 with transition value = %s \\n" %(transition_value)))
  input_value = randint(0,1)
  time.sleep(1)
  print("...Evaluating...")
  if (input_value == 0):
    result = yield from State3(input_value)
  else :
    result = yield from State2(input_value)
  result = "State 1 calling " + result
  return (outputValue + str(result))

@asyncio.coroutine
def State2(transition_value):
  outputValue = str(("State 2 with transition value = %s \\n"  %(transition_value)))
  input_value = randint(0,1)
  time.sleep(1)
  print("...Evaluating...")
  if (input_value == 0):
    result = yield from State1(input_value)
  else :
    result = yield from State3(input_value)
  result = "State 2 calling " + result
  return (outputValue + str(result))

@asyncio.coroutine
def State3(transition_value):
  outputValue = str(("State 3 with transition value = %s \\n" %(transition_value)))
  input_value = randint(0,1)
  time.sleep(1)
  print("...Evaluating...")
  if (input_value == 0):
    result = yield from State1(input_value)
  else :
    result = yield from EndState(input_value)
  result = "State 3 calling " + result
  return (outputValue + str(result))

@asyncio.coroutine
def EndState(transition_value):
  outputValue = str(("End State with transition value = %s \\n" %(transition_value)))
  print("...Stop Computation...")
  return (outputValue )


if __name__ == "__main__":
  print("Finite State Machine simulation with Asyncio Coroutine")
  loop = asyncio.get_event_loop()
  loop.run_until_complete(StartState())

  """
  
# asyncio_state_machine.py
with open('/content/asyncio_state_machine.py', 'w') as f:
  f.write(asyncio_state_machine.__doc__)


!python '/content/asyncio_state_machine.py'

Finite State Machine simulation with Asyncio Coroutine
Start State called 

...Evaluating...
...Evaluating...
...Evaluating...
...Evaluating...
...Evaluating...
...Evaluating...
...Evaluating...
...Evaluating...
...Stop Computation...
Resume of the Transition : 
Start State calling State 1 with transition value = 1 
State 1 calling State 2 with transition value = 1 
State 2 calling State 1 with transition value = 0 
State 1 calling State 2 with transition value = 1 
State 2 calling State 3 with transition value = 1 
State 3 calling State 1 with transition value = 0 
State 1 calling State 2 with transition value = 1 
State 2 calling State 3 with transition value = 1 
State 3 calling End State with transition value = 1 



**Task manipulation with Asyncio**

Asyncio предназначен для обработки асинхронных процессов и одновременного выполнения задач в цикле
событий (event loop). Для того, чтобы использовать задачу в корутине и обеспечить асинхронность используется класс Task() для обертывания. Это позволяет независимо выполняемым задачам выполняться одновременно с
другими задачами в том же цикле событий. При обёртывании корутины в задачу, она подключает
задачу к циклу событий, а затем автоматически запускается при старте цикла, обеспечивая тем самым
механизм автоматического управления корутиной.

In [ ]:
def asyncio_Task():
  """
import asyncio

@asyncio.coroutine
def factorial(number):
  f = 1
  for i in range(2, number+1):
    print("Asyncio.Task: Compute factorial(%s)" % (i))
    yield from asyncio.sleep(1)
    f *= i
  print("Asyncio.Task - factorial(%s) = %s" % (number, f))

@asyncio.coroutine
def fibonacci(number):
  a, b = 0, 1
  for i in range(number):
    print("Asyncio.Task: Compute fibonacci (%s)" % (i))
    yield from asyncio.sleep(1)
    a, b = b, a + b
  print("Asyncio.Task - fibonacci(%s) = %s" % (number, a))


@asyncio.coroutine
def binomialCoeff(n, k):
  result = 1
  for i in range(1, k+1):
    result = result * (n-i+1) / i
    print("Asyncio.Task: Compute binomialCoeff (%s)" % (i))
    yield from asyncio.sleep(1)
  print("Asyncio.Task - binomialCoeff(%s , %s) = %s" % (n,k,result))


if __name__ == "__main__":
  tasks = [asyncio.Task(factorial(10)), asyncio.Task(fibonacci(10)), asyncio.Task(binomialCoeff(20,10))]
  loop = asyncio.get_event_loop()
  loop.run_until_complete(asyncio.wait(tasks))
  loop.close()
  """

# asyncio_Task.py
with open('/content/asyncio_Task.py', 'w') as f:
  f.write(asyncio_Task.__doc__)


!python '/content/asyncio_Task.py'



Asyncio.Task: Compute factorial(2)
Asyncio.Task: Compute fibonacci (0)
Asyncio.Task: Compute binomialCoeff (1)
Asyncio.Task: Compute factorial(3)
Asyncio.Task: Compute fibonacci (1)
Asyncio.Task: Compute binomialCoeff (2)
Asyncio.Task: Compute factorial(4)
Asyncio.Task: Compute fibonacci (2)
Asyncio.Task: Compute binomialCoeff (3)
Asyncio.Task: Compute factorial(5)
Asyncio.Task: Compute fibonacci (3)
Asyncio.Task: Compute binomialCoeff (4)
Asyncio.Task: Compute factorial(6)
Asyncio.Task: Compute fibonacci (4)
Asyncio.Task: Compute binomialCoeff (5)
Asyncio.Task: Compute factorial(7)
Asyncio.Task: Compute fibonacci (5)
Asyncio.Task: Compute binomialCoeff (6)
Asyncio.Task: Compute factorial(8)
Asyncio.Task: Compute fibonacci (6)
Asyncio.Task: Compute binomialCoeff (7)
Asyncio.Task: Compute factorial(9)
Asyncio.Task: Compute fibonacci (7)
Asyncio.Task: Compute binomialCoeff (8)
Asyncio.Task: Compute factorial(10)
Asyncio.Task: Compute fibonacci (8)
Asyncio.Task: Compute binomialCoeff (9)


**Dealing with Asyncio and Futures**

Ключевым компонентом модуля Asyncio является класс Future. Это очень похоже на
concurrent.futures.Futures, адаптированный для механизма
event_loop Asyncio. Асинхронный класс Future предоставляет результат, который еще не доступен, т.е. это некоторая абстракцию чего-то, что
еще предстоит выполнить.


In [ ]:
def Asyncio_future():
  """
import asyncio
import sys

#SUM OF N INTEGERS
@asyncio.coroutine
def first_coroutine(future,N):
  count = 0
  for i in range(1,N+1):
    count=count + i
  yield from asyncio.sleep(4)
  future.set_result("first coroutine (sum of N integers) result = "  + str(count))

#FACTORIAL(N)
@asyncio.coroutine
def second_coroutine(future,N):
  count = 1
  for i in range(2, N+1):
    count *= i
  yield from asyncio.sleep(3)
  future.set_result("second coroutine (factorial) result = " + str(count))

def got_result(future):
  print(future.result())

if __name__ == "__main__":
  N1 = int(sys.argv[1])
  N2 = int(sys.argv[2])

  loop = asyncio.get_event_loop()
  future1 = asyncio.Future()
  future2 = asyncio.Future()

  tasks = [
    first_coroutine(future1,N1),
    second_coroutine(future2,N2)]

  future1.add_done_callback(got_result)
  future2.add_done_callback(got_result)

  loop.run_until_complete(asyncio.wait(tasks))
  loop.close()
  """

 # Asyncio_future.py
with open('/content/Asyncio_future.py', 'w') as f:
  f.write(Asyncio_future.__doc__)

In [ ]:
!python '/content/Asyncio_future.py'  1 1

second coroutine (factorial) result = 1
first coroutine (sum of N integers) result = 1


In [ ]:
!python '/content/Asyncio_future.py'  2 2 

second coroutine (factorial) result = 2
first coroutine (sum of N integers) result = 3


In [ ]:
!python '/content/Asyncio_future.py'  3 3

second coroutine (factorial) result = 6
first coroutine (sum of N integers) result = 6


In [ ]:
!python '/content/Asyncio_future.py'  5 5 

second coroutine (factorial) result = 120
first coroutine (sum of N integers) result = 15


In [ ]:
!python '/content/Asyncio_future.py'  1 10

second coroutine (factorial) result = 3628800
first coroutine (sum of N integers) result = 1


**Выводы**

В данной лабораторной работе была рассмотрена концепцию асинхронного программирования для параллельного выполнения вычислительных задач.

В данной работе рассмотрена работа с  библиотеками concurrent.futures, позволяющая работать с пулом потоков и процессов, а также с библиотекой  asyncio, позволяющая выполнять код асинхронно. 

Асинхронное программирование убирает блокирующую операцию из основного потока программы разделяя программу на отдельные единицы выполнения, работающие параллельно от основного потока или процесса. Некоторые способы организации асинхронного кода были представлены в книге: цикл событий(event loop), корутины(coroutines), futures. 